In [2]:
import numpy as np
import csv
import pandas as pd

In [3]:
#loading merged final data prepared using Knime
merged_file = 'C:/Users/Data/Final_File_with SJR.xlsx'

# Load spreadsheet
x3 = pd.ExcelFile(merged_file)
print(x3.sheet_names)
  

['final_file']


In [4]:
df_merged3 = x3.parse('final_file')#parse

In [5]:
df_merged3.head()#structure of the data

,Journal,Article,Author,Author Number,SJR
0,Molecular Autism,Statistical analysis of twenty years (1993 to ...,"Zhou, Wei-Zhen",1,2.753
1,Molecular Autism,Statistical analysis of twenty years (1993 to ...,"Ye, Adam Yongxin",2,2.753
2,Molecular Autism,Statistical analysis of twenty years (1993 to ...,"Sun, Zhong-Kai",3,2.753
3,Molecular Autism,Gene expression analysis of human induced plur...,"Germain, Noelle D",1,2.753
4,Molecular Autism,Gene expression analysis of human induced plur...,"Chen, Pin-Fang",2,2.753


In [6]:
#as we need to identify total score an dthen top 10 scores for authors. We need to identify unique author names first
#storing distinct author names in an array
Distinct_author = df_merged3['Author'].unique()
Distinct_author

array(['Zhou, Wei-Zhen', 'Ye, Adam Yongxin', 'Sun, Zhong-Kai', ...,
       'Bal, Matthieu', 'Zaorsky, Nicholas G', 'Zhou, Fei'], dtype=object)

In [7]:
#grouping by and count on Author Number to calculate ranking furher using SJR
df3_group_1 = pd.DataFrame(df_merged3.groupby(['Author', 'Article','Journal','Author Number','SJR'])['Author Number'].count())
(df3_group_1.head())

,,,,,Author Number
Author,Article,Journal,Author Number,SJR,
"Aagaard, Kjersti",Microbiome: Antibiotics and the infant microflora.,Nature Reviews Microbiology,2,16.373,1
"Abate-Shen, Cory",Optimizing mouse models for precision cancer prevention.,Nature Reviews Cancer,3,25.467,1
"Abdel-Wahab, Omar",RNA splicing factors as oncoproteins and tumour suppressors.,Nature Reviews Cancer,3,25.467,1
"Abrahams, Katherine A",THPP target assignment reveals EchA6 as an essential fatty acid shuttle in mycobacteria.,Nature Reviews Microbiology,2,16.373,1
"Abt, Birte",Corrigendum: The Mouse Intestinal Bacterial Collection (miBC) provides host-specific insight into cultured diversity and functional potential of the gut microbiota.,Nature Reviews Microbiology,3,16.373,1


In [8]:
# to avoid confusion the column is set as Article count
df3_group_1.columns = ['Article Count']


In [9]:
#resetting the index
df3_group_1.reset_index(inplace=True)
df3_group_1.head()
len(df3_group_1)

3869

In [10]:
#calculating ranking based on the gievn default formula
#first creating data frame for total score of each author
#after that we us efor loop to calculate ranking of each author.
overall_sjr_ranking = pd.DataFrame(data=None)
for author in Distinct_author:
    ranking = 0
    #df_merged3
    first_rank = df3_group_1[df3_group_1['Author'] == author]; first_rank = first_rank[first_rank['Author Number'] == 1]
    ranking = first_rank['SJR'].max() if len(first_rank) >=1 else ranking
    second_rank = df3_group_1[df3_group_1['Author'] == author]; second_rank = second_rank[second_rank['Author Number'] == 2]
    ranking = second_rank['SJR'].max() if len(second_rank) >=1 else ranking
    third_rank = df3_group_1[df3_group_1['Author'] == author]; third_rank = third_rank[third_rank['Author Number'] == 3]
    ranking = third_rank['SJR'].max() if len(third_rank) >=1 else ranking
    ranking = (ranking* len(first_rank)) + (ranking* len(second_rank) * (1/3)) + (ranking* len(third_rank) * (1/3)) 
   
    overall_sjr_ranking = overall_sjr_ranking.append(pd.DataFrame(data=[[author, ranking]], columns=['Authors','Author Ranking']))
    overall_sjr_ranking.sort_values('Author Ranking', ascending=False)   

In [118]:
#Top 10 ranking of authors
overall_sjr_ranking.sort_values('Author Ranking', ascending=False).head(10)

,Authors,Author Ranking
0,"Zlotorynski, Eytan",1843.968
0,"Seton-Rogers, Sarah",1536.509
0,"Baumann, Kim",1492.736
0,"Strzyz, Paulina",856.128
0,"Wrighton, Katharine H",856.128
0,"Alderton, Gemma K",814.944
0,"Minton, Kirsty",790.272
0,"Villanueva, M Teresa",687.609
0,"Du Toit, Andrea",625.632
0,"McCarthy, Nicola",585.741


In [11]:
 overall_sjr_ranking =overall_sjr_ranking.sort_values('Author Ranking', ascending=False) 

In [12]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:/Users/Data/all_author_score.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
overall_sjr_ranking.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()